In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
import copy
from SBRP_DI.schuijbroek17 import visu, algorithms, tools, analytics

In [2]:
year = 2021
window_start = '{}-05-01 00:00:00'.format(year)
window_end = '{}-08-01 00:00:00'.format(year)

## Load the trip data

In [3]:
trips_dfs = []
months = ['04', '05', '06', '07', '08', '09', '10']
for month in months:
    trips_dfs.append(pd.read_csv('trips/2021-{}.csv'.format(month))) 
trips = pd.concat(trips_dfs)

In [4]:
trips.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2021-04-30T23:59:54,2021-05-01T00:08:15,16,Liisanpuistikko,2,Laivasillankatu,1885.0,499.0
1,2021-04-30T23:59:52,2021-05-01T00:16:16,20,Kaisaniemenpuisto,112,Rautatieläisenkatu,3359.0,984.0
2,2021-04-30T23:59:52,2021-05-01T00:13:40,4,Viiskulma,33,Kauppakorkeakoulu,1633.0,824.0
3,2021-04-30T23:59:43,2021-05-01T00:23:22,133,Paavalinpuisto,241,Agronominkatu,6459.0,1418.0
4,2021-04-30T23:59:41,2021-05-01T00:16:15,20,Kaisaniemenpuisto,112,Rautatieläisenkatu,3210.0,990.0


In [5]:
trips = analytics.preprocessTrips_helsinki(trips)

Determine time window of trips

In [6]:
trips = trips[window_start:window_end]

Get unique stations in the training set

In [7]:
unique_stations = trips['Start station number'].unique()

In [8]:
len(unique_stations)

459

## Load the station data

In [9]:
stations = pd.read_csv('locations/Helsingin_ja_Espoon_kaupunkipyöräasemat_2021.csv')

In [10]:
stations.head()

,FID,ID,Nimi,Namn,Name,Osoite,Adress,Kaupunki,Stad,Operaattor,Kapasiteet,x,y
0,1,501,Hanasaari,Hanaholmen,Hanasaari,Hanasaarenranta 1,Hanaholmsstranden 1,Espoo,Esbo,CityBike Finland,10,24.840319,60.165820
1,2,503,Keilalahti,Kägelviken,Keilalahti,Keilalahdentie 2,Kägelviksvägen 2,Espoo,Esbo,CityBike Finland,28,24.827467,60.171524
2,3,505,Westendinasema,Westendstationen,Westendinasema,Westendintie 1,Westendvägen 1,Espoo,Esbo,CityBike Finland,16,24.805758,60.168266
3,4,507,Golfpolku,Golfstigen,Golfpolku,Golfpolku 3,Golfstigen 3,Espoo,Esbo,CityBike Finland,16,24.796136,60.168143
4,5,509,Revontulentie,Norrskensvägen,Revontulentie,Revontulentie 10,Norrskensvägen 10,Espoo,Esbo,CityBike Finland,30,24.802938,60.171551


In [11]:
stations = analytics.preprocessStations_helsinki(stations)
stations['City'].replace(' ', 'Helsinki', inplace=True)

In [12]:
stations.shape

(457, 5)

In [13]:
# missing stations from stations
missing = unique_stations[~np.isin(unique_stations, stations.index)]
print('Missing stations:', missing)

Missing stations: [997 999 754]


In [14]:
# number of trips starting or ending at missing stations
trips[trips['Start station number'].isin(missing) | trips['End station number'].isin(missing)].shape

(1321, 7)

In [15]:
trips[trips['Start station number'].isin([754]) | trips['End station number'].isin([754])].shape

(1194, 7)

In [16]:
lintumetsa = pd.DataFrame({'Station ID': 754,
'Station': 'Lintumetsä',
'Longitude': 24.821200,
'Latitude': 60.238410,
'# of Docks' : 30,
'City' : 'Espoo'}, index=[1])
lintumetsa.set_index('Station ID', inplace=True)

stations = pd.concat([stations, lintumetsa])
stations.sort_index(inplace=True)

In [17]:
stations.to_csv('locations/{}.csv'.format(year))